In [424]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

df1 = pd.read_excel('/Users/seanyu_mac/Downloads/20241023_2300/원부_이미지_성분인식_결합_split_1.xlsx')
df2 = pd.read_excel('/Users/seanyu_mac/Downloads/20241023_2300/원부_이미지_성분인식_결합_split_2.xlsx')
df3 = pd.read_excel('/Users/seanyu_mac/Downloads/20241023_2300/원부_이미지_성분인식_결합_split_3.xlsx')
df4 = pd.read_excel('/Users/seanyu_mac/Downloads/20241023_2300/원부_이미지_성분인식_결합_split_4.xlsx')

df = pd.concat([df1, df2, df3, df4])

In [426]:
original_df = df
df.columns

Index(['group_id', 'seq_id', 'type', 'product_id', 'opr', 'opr_reason',
       'pb_exported', 'pname', 'img_url', 'score', 'nutri', 'ug_err', 'no_std',
       'big_cont', 'cont_err', 'std_pos', 'ambig_ingr', 'col_pos', 'std_fail',
       'cont_kg', 'ingr_fail', 'num_cont', 'vitab', 'dup', 'ug_ingr', 'url'],
      dtype='object')

In [340]:
df = original_df

In [428]:
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162390 entries, 0 to 162389
Data columns (total 26 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   group_id     162390 non-null  object 
 1   seq_id       162389 non-null  float64
 2   type         162390 non-null  object 
 3   product_id   162389 non-null  object 
 4   opr          162387 non-null  object 
 5   opr_reason   45058 non-null   object 
 6   pb_exported  162387 non-null  object 
 7   pname        162389 non-null  object 
 8   img_url      21041 non-null   object 
 9   score        162390 non-null  float64
 10  nutri        21041 non-null   object 
 11  ug_err       162390 non-null  int64  
 12  no_std       162390 non-null  int64  
 13  big_cont     162390 non-null  int64  
 14  cont_err     162390 non-null  int64  
 15  std_pos      162390 non-null  int64  
 16  ambig_ingr   162390 non-null  int64  
 17  col_pos      162390 non-null  int64  
 18  std_fail     162390 non-

In [430]:
df = df[df['nutri'].isna() == False]
df.drop(df[df['nutri'] == '[]'].index, inplace=True)
df.drop(df[df['nutri'] == 0].index, inplace=True)
len(df.index)

/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_30566/2021006235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['nutri'] == '[]'].index, inplace=True)
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_30566/2021006235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['nutri'] == 0].index, inplace=True)


19085

In [346]:
df = df.drop(df.loc[df['group_id'].duplicated()].index)
len(df.index)

15745

In [464]:
df = df.drop(df.loc[df['pname'].duplicated()].index)
len(df.index)

15702

In [350]:
df.drop(df[df['pb_exported'] == 'Y'].index, inplace=True)
len(df.index)

14293

In [352]:
df.reset_index(drop=True, inplace=True)

In [360]:
nutritions = []

for i in df['nutri'].tolist():
    i = json.loads(i)
    nutritions.append(i)

df['nutrition'] = nutritions
df.loc[0, 'nutrition']

[{'content': '0kcal', 'ingredient': '열량', 'nutri_standards': None},
 {'ingredient': '탄수화물', 'content': '1g미만', 'nutri_standards': '0%'},
 {'nutri_standards': '0%', 'content': '0g', 'ingredient': '단백질'},
 {'nutri_standards': '0%', 'content': '0g', 'ingredient': '지방'},
 {'content': '0mg', 'ingredient': '나트륨', 'nutri_standards': '0%'},
 {'ingredient': '실리마린', 'content': '130mg', 'nutri_standards': None},
 {'nutri_standards': '100%', 'content': '1.2mg', 'ingredient': '비타민b1'},
 {'nutri_standards': '100%', 'ingredient': '비타민b2', 'content': '1.4mg'},
 {'nutri_standards': '100%', 'ingredient': '비타민b6', 'content': '1.5mg'}]

In [366]:
nutris = []

for i in df['nutrition']:
    data_list = []
    for j in range(len(i)):
        data_dict = i[j]
        content = data_dict['content']
        ingredient = data_dict['ingredient']
        nutri_standards = data_dict['nutri_standards']
        data_str = f'{ingredient} : {content} ({nutri_standards})'
        data_list.append(data_str)
    nutris.append(data_list)

In [374]:
df['nutrition'] = nutris
df.to_csv('/Users/seanyu_mac/Downloads/큐리온중복제거_20241105.csv')